# Part 2: Using the features/factors you take and discovered, e.g., FRED, Fama-French website, ADS, AR, CAPM, momentum factors, volume, price/return lags, etc.) to construct a feature database
- The target variable Y can be either price or return
- Frequency could be either daily or monthly

In [203]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import ta
from fredapi import Fred

# Common Features

In [204]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 8, 31)
NFLX_STOCK = yf.download("NFLX", start_date, end_date)

NFLX_STOCK.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,326.100006,329.980011,324.779999,329.809998,329.809998,4485800
2020-01-03,326.779999,329.859985,325.529999,325.899994,325.899994,3806900
2020-01-06,323.119995,336.359985,321.200012,335.829987,335.829987,5663100
2020-01-07,336.470001,336.700012,330.299988,330.750000,330.750000,4703200
2020-01-08,331.489990,342.700012,331.049988,339.260010,339.260010,7104500


# FRED

In [205]:
fred = Fred(api_key='8f2ffb8bdd18027904612e1c689af3f1')
varList = ['T10Y3M', 'DGS10', 'OBMMIJUMBO30YF',  # term premium 10yr-3mon, 30 yr mortgage jumbo loan
           'DEXUSEU', 'DEXJPUS', 'DEXUSUK', # spot exchange rates to EUR, JPY, GBP 
           'CBBTCUSD', 'CBETHUSD',  # cryptocurrencies
               'T10YIE', 'DCOILBRENTEU', # breakeven inflation + brent oil price 
               'VIXCLS', # implied volatilities
               'DAAA', 'DBAA', # corporate bond yield
              'AMERIBOR', 'T5YIE', 'BAMLH0A0HYM2','BAMLH0A0HYM2EY', 'DGS1', 'DCOILWTICO', 
                              'DHHNGSP'] 

SP500 = fred.get_series('SP500')
SP500.name = 'SP500'
df_fred = SP500

# merge data series
for i in range(0, len(varList)):
    data = fred.get_series(varList[i])
    data.name = varList[i]
    df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)
    


In [206]:
df_fred.head()

,SP500,T10Y3M,DGS10,OBMMIJUMBO30YF,DEXUSEU,DEXJPUS,DEXUSUK,CBBTCUSD,CBETHUSD,T10YIE,...,VIXCLS,DAAA,DBAA,AMERIBOR,T5YIE,BAMLH0A0HYM2,BAMLH0A0HYM2EY,DGS1,DCOILWTICO,DHHNGSP
2017-01-03,2257.83,1.92,2.45,4.432,1.0416,117.68,1.2256,1020.67,9.59,1.98,...,12.85,3.94,4.72,0.79431,1.86,4.13,6.12,0.89,52.36,3.41
2017-01-04,2270.75,1.93,2.46,4.355,1.0476,117.38,1.2299,1130.30,11.17,1.99,...,11.85,3.93,4.71,0.78633,1.86,4.02,6.02,0.87,53.26,3.42
2017-01-05,2269.00,1.85,2.37,4.377,1.0598,115.46,1.2415,1007.00,10.23,1.95,...,11.67,3.85,4.62,0.78379,1.83,4.04,5.96,0.83,53.77,3.42
2017-01-06,2276.98,1.89,2.42,4.385,1.0560,116.85,1.2314,895.71,10.14,1.96,...,11.32,3.91,4.66,0.81205,1.84,3.98,5.95,0.85,53.98,3.38
2017-01-09,2268.90,1.88,2.38,4.302,1.0576,116.07,1.2167,902.66,10.39,1.95,...,11.56,3.88,4.62,0.78569,1.85,4.02,5.95,0.82,51.95,3.14


In [207]:
NFLX_STOCK = pd.concat([NFLX_STOCK, df_fred], axis=1)
NFLX_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,SP500,T10Y3M,DGS10,OBMMIJUMBO30YF,...,VIXCLS,DAAA,DBAA,AMERIBOR,T5YIE,BAMLH0A0HYM2,BAMLH0A0HYM2EY,DGS1,DCOILWTICO,DHHNGSP
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,2257.83,1.92,2.45,4.432,...,12.85,3.94,4.72,0.79431,1.86,4.13,6.12,0.89,52.36,3.41
2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,2270.75,1.93,2.46,4.355,...,11.85,3.93,4.71,0.78633,1.86,4.02,6.02,0.87,53.26,3.42
2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,2269.00,1.85,2.37,4.377,...,11.67,3.85,4.62,0.78379,1.83,4.04,5.96,0.83,53.77,3.42
2017-01-06,NaN,NaN,NaN,NaN,NaN,NaN,2276.98,1.89,2.42,4.385,...,11.32,3.91,4.66,0.81205,1.84,3.98,5.95,0.85,53.98,3.38
2017-01-09,NaN,NaN,NaN,NaN,NaN,NaN,2268.90,1.88,2.38,4.302,...,11.56,3.88,4.62,0.78569,1.85,4.02,5.95,0.82,51.95,3.14


# Fama-French

In [208]:
data_ff5 = pd.read_csv('resources/F-F_Research_Data_5_Factors_2x3_daily.csv')
data_ff5.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data_ff5["Date"] = pd.to_datetime(data_ff5["Date"], format="%Y%m%d")
data_ff5 = data_ff5[(data_ff5["Date"] >= start_date) & (data_ff5["Date"] <= end_date)]
data_ff5 = data_ff5.set_index('Date')
data_ff5

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2020-01-02,0.86,-0.97,-0.33,0.24,-0.22,0.006
2020-01-03,-0.67,0.30,0.00,-0.13,-0.11,0.006
2020-01-06,0.36,-0.21,-0.55,-0.17,-0.26,0.006
2020-01-07,-0.19,-0.03,-0.25,-0.13,-0.24,0.006
2020-01-08,0.47,-0.16,-0.66,-0.16,-0.18,0.006
...,...,...,...,...,...,...
2024-08-26,-0.34,0.33,0.17,0.13,-0.06,0.022
2024-08-27,0.05,-0.90,0.02,0.27,0.23,0.022
2024-08-28,-0.67,-0.22,1.14,0.55,-0.16,0.022


# ADS

In [209]:
ads = pd.read_excel("resources/ADS_Index_Most_Current_Vintage.xlsx")
ads.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
ads["Date"] = pd.to_datetime(ads["Date"], format="%Y:%m:%d")
ads = ads[(ads["Date"] >= start_date) & (ads["Date"] <= end_date)]
ads = ads.set_index("Date")
ads

,ADS_Index
Date,
2020-01-01,-0.301644
2020-01-02,-0.289801
2020-01-03,-0.278009
2020-01-04,-0.266263
2020-01-05,-0.254558
...,...
2024-08-27,-0.114812
2024-08-28,-0.119569
2024-08-29,-0.125109


# Momentum Factors

### 1.Rate of Change (ROC)

In [195]:
NFLX_STOCK['ROC_10'] = NFLX_STOCK['Close'].pct_change(periods=10) * 100


C:\Users\ankit\AppData\Local\Temp\ipykernel_10152\3275680082.py:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  NFLX_STOCK['ROC_10'] = NFLX_STOCK['Close'].pct_change(periods=10) * 100


### 2. Relative Strength Index (RSI)

In [196]:
def compute_RSI(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

NFLX_STOCK['RSI_14'] = compute_RSI(NFLX_STOCK)

### 3. Moving Average Convergence Divergence (MACD)

In [132]:
NFLX_STOCK['EMA_12'] = NFLX_STOCK['Close'].ewm(span=12, min_periods=0).mean()
NFLX_STOCK['EMA_26'] = NFLX_STOCK['Close'].ewm(span=26, min_periods=0).mean()
NFLX_STOCK['MACD'] = NFLX_STOCK['EMA_12'] - NFLX_STOCK['EMA_26']
NFLX_STOCK['Signal_Line'] = NFLX_STOCK['MACD'].ewm(span=9, min_periods=0).mean()


### 4. Average True Range (ATR)

In [133]:
NFLX_STOCK['High-Low'] = NFLX_STOCK['High'] - NFLX_STOCK['Low']
NFLX_STOCK['High-Close'] = abs(NFLX_STOCK['High'] - NFLX_STOCK['Close'].shift(1))
NFLX_STOCK['Low-Close'] = abs(NFLX_STOCK['Low'] - NFLX_STOCK['Close'].shift(1))
NFLX_STOCK['TR'] = NFLX_STOCK[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
NFLX_STOCK['ATR_14'] = NFLX_STOCK['TR'].rolling(window=14).mean()


### 5. Momentum Indicator

In [197]:
NFLX_STOCK['Momentum_10'] = NFLX_STOCK['Close'] - NFLX_STOCK['Close'].shift(10)


# Volume Indicator

### 1. On-Balance Volume (OBV)

In [170]:
NFLX_STOCK['OBV'] = (np.sign(NFLX_STOCK['Close'].diff()) * NFLX_STOCK['Volume']).cumsum()


### 2. Volume-Weighted Average Price (VWAP)

In [171]:
NFLX_STOCK['Typical_Price'] = (NFLX_STOCK['High'] + NFLX_STOCK['Low'] + NFLX_STOCK['Close']) / 3
NFLX_STOCK['Cumulative_TPV'] = (NFLX_STOCK['Typical_Price'] * NFLX_STOCK['Volume']).cumsum()
NFLX_STOCK['Cumulative_Volume'] = NFLX_STOCK['Volume'].cumsum()
NFLX_STOCK['VWAP'] = NFLX_STOCK['Cumulative_TPV'] / NFLX_STOCK['Cumulative_Volume']


# Trend Indicators

### 1.  Simple Moving Average (SMA)

In [198]:
NFLX_STOCK['SMA_50'] = NFLX_STOCK['Close'].rolling(window=50).mean()
NFLX_STOCK['SMA_200'] = NFLX_STOCK['Close'].rolling(window=200).mean()


### 2. Exponential Moving Average (EMA)

In [199]:
NFLX_STOCK['EMA_50'] = NFLX_STOCK['Close'].ewm(span=50).mean()
NFLX_STOCK['EMA_200'] = NFLX_STOCK['Close'].ewm(span=200).mean()


# Volatility Indicators

### 1. Bollinger Bands

In [140]:

NFLX_STOCK['BB_Upper'] = NFLX_STOCK['BB_Mid'] + 2 * NFLX_STOCK['Close'].rolling(window=20).std()
NFLX_STOCK['BB_Lower'] = NFLX_STOCK['BB_Mid'] - 2 * NFLX_STOCK['Close'].rolling(window=20).std()


KeyError: 'BB_Mid'

### 2.  Average True Range (ATR)

In [141]:
NFLX_STOCK['High-Low'] = NFLX_STOCK['High'] - NFLX_STOCK['Low']
NFLX_STOCK['High-Close'] = abs(NFLX_STOCK['High'] - NFLX_STOCK['Close'].shift(1))
NFLX_STOCK['Low-Close'] = abs(NFLX_STOCK['Low'] - NFLX_STOCK['Close'].shift(1))
NFLX_STOCK['TR'] = NFLX_STOCK[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
NFLX_STOCK['ATR_14'] = NFLX_STOCK['TR'].rolling(window=14).mean()
NFLX_STOCK.head()



,Open,High,Low,Close,Adj Close,Volume,SP500_x,T10Y3M,DGS10,DGS1,...,Momentum_10,OBV,Typical_Price,Cumulative_TPV,Cumulative_Volume,VWAP,SMA_50,SMA_200,EMA_50,EMA_200
2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-01,NaN,NaN,NaN,NaN,NaN,NaN,1939.38,1.62,1.97,0.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-03-01,NaN,NaN,NaN,NaN,NaN,NaN,1978.35,1.50,1.83,0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,2072.78,1.56,1.79,0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-01,NaN,NaN,NaN,NaN,NaN,NaN,2099.33,1.55,1.85,0.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Volatility Index (VIX)

In [118]:

VIX = yf.download("^VIX", start_date, end_date)
NFLX_STOCK['VIX_Close'] = VIX['Close']


[*********************100%%**********************]  1 of 1 completed


In [200]:
NFLX_STOCK.dropna(inplace=True)
NFLX_STOCK.head()


,Open,High,Low,Close,Adj Close,Volume,SP500,T10Y3M,DGS10,OBMMIJUMBO30YF,...,DGS1,DCOILWTICO,DHHNGSP,ROC_10,RSI_14,Momentum_10,SMA_50,SMA_200,EMA_50,EMA_200


### Returns and Daily Returns

In [ ]:
NFLX_STOCK["Price Momentum"] = NFLX_STOCK["Adj Close"] - NFLX_STOCK["Adj Close"].shift(1) # difference
NFLX_STOCK["Return"] = np.log(NFLX_STOCK["Adj Close"]) - np.log(NFLX_STOCK["Adj Close"].shift(1)) # percentage
NFLX_STOCK.dropna(inplace=True)
NFLX_STOCK.head()

NameError: name 'NFLX_STOCK' is not defined

In [211]:
filename = f"NFLX_Feature_Mart.csv"
NFLX_STOCK.dropna(inplace=True)
NFLX_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,SP500,T10Y3M,DGS10,OBMMIJUMBO30YF,...,DBAA,AMERIBOR,T5YIE,BAMLH0A0HYM2,BAMLH0A0HYM2EY,DGS1,DCOILWTICO,DHHNGSP,Returns,Daily_Return
2020-01-03,326.779999,329.859985,325.529999,325.899994,325.899994,3806900.0,3234.85,0.28,1.80,3.908,...,3.79,1.599492,1.69,3.61,5.25,1.55,63.00,2.06,-3.910004,-0.011926
2020-01-06,323.119995,336.359985,321.200012,335.829987,335.829987,5663100.0,3246.28,0.25,1.81,3.944,...,3.83,1.599941,1.69,3.57,5.23,1.54,63.27,2.10,9.929993,0.030014
2020-01-07,336.470001,336.700012,330.299988,330.750000,330.750000,4703200.0,3237.18,0.29,1.83,4.012,...,3.86,1.597702,1.67,3.54,5.21,1.53,62.70,2.17,-5.079987,-0.015242
2020-01-08,331.489990,342.700012,331.049988,339.260010,339.260010,7104500.0,3253.05,0.33,1.87,3.889,...,3.92,1.586618,1.68,3.50,5.22,1.55,59.65,2.09,8.510010,0.025404
2020-01-09,342.000000,343.420013,334.609985,335.660004,335.660004,4709300.0,3274.70,0.31,1.85,4.054,...,3.88,1.588611,1.67,3.49,5.19,1.54,59.56,2.05,-3.600006,-0.010668


In [212]:
NFLX_STOCK.to_csv(filename, index=True)

In [94]:
NFLX_STOCK.to_csv(filename, index=True)